In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [2]:
list=[[324.456 , 'hi' ,    'test'],
[453.987  ,'hello',  'python'],
[768.66   ,'test',   'java']]

df=spark.createDataFrame(list,['col1','col2','col3'])

df.printSchema()

root
-- col1: double (nullable = true)
-- col2: string (nullable = true)
-- col3: string (nullable = true)

In [3]:
df.withColumn("col4",F.expr("""substring(col1,0,instr(col1),"."+2""")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling z:org.apache.spark.sql.functions.expr.
: org.apache.spark.sql.catalyst.parser.ParseException: 
extraneous input '<EOF>' expecting {')', ','}(line 1, pos 34)

== SQL ==
substring(col1,0,instr(col1),"."+2
----------------------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parseExpression(ParseDriver.scala:44)
	at com.databricks.sql.parser.DatabricksSqlParser.parseExpression(DatabricksSqlParser.scala:46)
	at org.apache.spark.sql.functions$.expr(functions.scala:1366)
	at org.apache.spark.sql.functions.expr(functions.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-2240815886526342> in <module> 
 ----> 1 df . withColumn ( "col4" , F . expr ( """substring(col1,0,instr(col1),"."+2""" ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in expr (str) 
 673 """
 674 sc = SparkContext . _active_spark_context
 --> 675 return Column ( sc . _jvm . functions . expr ( str ) ) 
 676 
 677 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : '\nextraneous input \'<EOF>\' expecting {\')\', \',\'}(line 1, pos 34)\n\n== SQL ==\nsubstring(col1,0,instr(col1),"."+2\n----------------------------------^^^\n'

In [4]:
df.withColumn('test',F.regexp_extract("col1",'\d+[.]\d{2}',0)).show()

+-------+-----+------+------+
 col1| col2| col3| test|
+-------+-----+------+------+
324.456| hi| test|324.45|
453.987|hello|python|453.98|
 768.66| test| java|768.66|
+-------+-----+------+------+

In [5]:
df.withColumn("col4", F.expr("""IF(length(col1)==7, substring(col1,0,length(col1)-1),col1)""")).show()

+-------+-----+------+------+
 col1| col2| col3| col4|
+-------+-----+------+------+
324.456| hi| test|324.45|
453.987|hello|python|453.98|
 768.66| test| java|768.66|
+-------+-----+------+------+

In [6]:
col1     col2   col3   col4
324.456  hi     test   324.45
453.987  hello  python 453.98
768.66   test   java   768.66

In [7]:
list=[['t1', ['m1']],
      ['t3' ,['m1', 'm2']],
      ['t4' , ['m1', 'm2']],
      ['t6' , ['m2']],
      ['t7' , ['m3']],
      ['t8' , ['m3']],
      ['t9' , ['m1']]]


df=spark.createDataFrame(list,['time','message'])

df.show()

+----+--------+
time| message|
+----+--------+
 t1| [m1]|
 t3|[m1, m2]|
 t4|[m1, m2]|
 t6| [m2]|
 t7| [m3]|
 t8| [m3]|
 t9| [m1]|
+----+--------+

In [8]:
w=Window().partitionBy("message")
df.withColumn("mono", F.monotonically_increasing_id())\
  .withColumn("message", F.explode("message"))\
  .show()
  

+----+-------+-----------+
time|message| mono|
+----+-------+-----------+
 t1| m1| 8589934592|
 t3| m1|17179869184|
 t3| m2|17179869184|
 t4| m1|25769803776|
 t4| m2|25769803776|
 t6| m2|34359738368|
 t7| m3|42949672960|
 t8| m3|51539607552|
 t9| m1|60129542144|
+----+-------+-----------+

In [9]:
+--------+-------+-----+
 message | start | end |
+--------+-------+-----+
   m1    |  t1   |  t4 |
   m2    |  t3   |  t6 |
   m3    |  t7   |  t8 |
   m1    |  t9   |  t9 |

In [10]:
list=[[1,'2015-01-05' , '2015-01-05',  0.0076],
     [1,'2015-01-06' , '2015-01-05',  0.0026],
     [1,'2015-01-07' , '2015-01-05',  0.0016],
     [1,'2015-01-08' , '2015-01-05',  0.0006],
     [2,'2015-01-09' , '2015-01-05',  0.0012],
     [2,'2015-01-10' , '2015-01-05',  0.0014],
     [1,'2015-01-12' , '2015-01-12',  0.0026],
     [1,'2015-01-13' , '2015-01-12',  0.0086],
     [1,'2015-01-14' , '2015-01-12',  0.0046],
     [1,'2015-01-15' , '2015-01-12',  0.0021],
     [2,'2015-01-16' , '2015-01-12',  0.0042],
     [2,'2015-01-17' , '2015-01-12',  0.0099]]

df=spark.createDataFrame(list,['id','calendarday','last_monday','indexCP'])


df.show()

+---+-----------+-----------+-------+
 id|calendarday|last_monday|indexCP|
+---+-----------+-----------+-------+
 1| 2015-01-05| 2015-01-05| 0.0076|
 1| 2015-01-06| 2015-01-05| 0.0026|
 1| 2015-01-07| 2015-01-05| 0.0016|
 1| 2015-01-08| 2015-01-05| 6.0E-4|
 2| 2015-01-09| 2015-01-05| 0.0012|
 2| 2015-01-10| 2015-01-05| 0.0014|
 1| 2015-01-12| 2015-01-12| 0.0026|
 1| 2015-01-13| 2015-01-12| 0.0086|
 1| 2015-01-14| 2015-01-12| 0.0046|
 1| 2015-01-15| 2015-01-12| 0.0021|
 2| 2015-01-16| 2015-01-12| 0.0042|
 2| 2015-01-17| 2015-01-12| 0.0099|
+---+-----------+-----------+-------+

In [11]:
w=Window().partitionBy("last_monday")\
          .orderBy(F.to_date("calendarday","yyyy-MM-dd"))\
          .rowsBetween(Window.unboundedPreceding,Window.currentRow)

df.withColumn("PreviousYearUnique", F.first("indexCP").over(w)).show()

#+---+-----------+-----------+-------+------------------+
#| id|calendarday|last_monday|indexCP|PreviousYearUnique|
#+---+-----------+-----------+-------+------------------+
#|  1| 2015-01-05| 2015-01-05| 0.0076|            0.0076|
#|  1| 2015-01-06| 2015-01-05| 0.0026|            0.0076|
#|  1| 2015-01-07| 2015-01-05| 0.0016|            0.0076|
#|  1| 2015-01-08| 2015-01-05| 6.0E-4|            0.0076|
#|  2| 2015-01-09| 2015-01-05| 0.0012|            0.0076|
#|  2| 2015-01-10| 2015-01-05| 0.0014|            0.0076|
#|  1| 2015-01-12| 2015-01-12| 0.0026|            0.0026|
#|  1| 2015-01-13| 2015-01-12| 0.0086|            0.0026|
#|  1| 2015-01-14| 2015-01-12| 0.0046|            0.0026|
#|  1| 2015-01-15| 2015-01-12| 0.0021|            0.0026|
#|  2| 2015-01-16| 2015-01-12| 0.0042|            0.0026|
#|  2| 2015-01-17| 2015-01-12| 0.0099|            0.0026|
#+---+-----------+-----------+-------+------------------+

+---+-----------+-----------+-------+------------------+
 id|calendarday|last_monday|indexCP|PreviousYearUnique|
+---+-----------+-----------+-------+------------------+
 1| 2015-01-05| 2015-01-05| 0.0076| 0.0076|
 1| 2015-01-06| 2015-01-05| 0.0026| 0.0076|
 1| 2015-01-07| 2015-01-05| 0.0016| 0.0076|
 1| 2015-01-08| 2015-01-05| 6.0E-4| 0.0076|
 2| 2015-01-09| 2015-01-05| 0.0012| 0.0076|
 2| 2015-01-10| 2015-01-05| 0.0014| 0.0076|
 1| 2015-01-12| 2015-01-12| 0.0026| 0.0026|
 1| 2015-01-13| 2015-01-12| 0.0086| 0.0026|
 1| 2015-01-14| 2015-01-12| 0.0046| 0.0026|
 1| 2015-01-15| 2015-01-12| 0.0021| 0.0026|
 2| 2015-01-16| 2015-01-12| 0.0042| 0.0026|
 2| 2015-01-17| 2015-01-12| 0.0099| 0.0026|
+---+-----------+-----------+-------+------------------+

In [12]:
df = spark.createDataFrame(
    [[1, 'foo'],
     [1, 'bar'], 
     [1, 'foo'], 
     [1, 'foo'], 
     [2, 'bar'], 
     [2, 'foo'], 
     [2, 'bar'], 
     [2, 'foo'],
     [3,'foo']],
    ['session_id', 'event'])
df.show()

+----------+-----+
session_id|event|
+----------+-----+
 1| foo|
 1| bar|
 1| foo|
 1| foo|
 2| bar|
 2| foo|
 2| bar|
 2| foo|
 3| foo|
+----------+-----+

In [13]:
df = spark.createDataFrame(
        [[1, '2020-01-01 12:30:00.000', 'foo'], [1, '2020-01-01 12:31:00.000', 'bar'], [1, '2020-01-01 12:32:00.000', 'foo'],
    [1, '2020-01-01 12:33:00.000', 'foo'], [2, '2020-01-01 13:00:00.000', 'bar'], [2, '2020-01-01 13:01:00.000', 'foo'],
    [2, '2020-01-01 13:02:00.000', 'bar'], [2, '2020-01-01 13:03:00.000', 'foo']],
        ['session_id', 'timestamp', 'event']
    )
df.show(truncate=False)

+----------+-----------------------+-----+
session_id|timestamp |event|
+----------+-----------------------+-----+
1 |2020-01-01 12:30:00.000|foo |
1 |2020-01-01 12:31:00.000|bar |
1 |2020-01-01 12:32:00.000|foo |
1 |2020-01-01 12:33:00.000|foo |
2 |2020-01-01 13:00:00.000|bar |
2 |2020-01-01 13:01:00.000|foo |
2 |2020-01-01 13:02:00.000|bar |
2 |2020-01-01 13:03:00.000|foo |
+----------+-----------------------+-----+

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("timestamp")
w2=Window().partitionBy("session_id").orderBy("timestamp")
w3=Window().partitionBy("session_id")
df.withColumn("timestamp", F.to_timestamp("timestamp", 'yyyy-MM-dd HH:mm:ss.SSS'))\
  .withColumn("session_id", F.sum(F.when((F.col("event")=='bar'),F.lit(1))\
                                         .otherwise(F.lit(0))).over(w))\
  .withColumn("rowNum", F.row_number().over(w2))\
  .withColumn("max", F.max("rowNum").over(w3))\
  .withColumn("first", F.when((F.col("rowNum")==1)&(F.col("event")=='foo'), F.lit(1))\
                       .otherwise(F.lit(0)))\
  .filter('max>=2 and first=0').drop(*['rowNum','sample_timestamp','max','first']).show()

#+----------+-------------------+-----+
#|session_id|          timestamp|event|
#+----------+-------------------+-----+
#|         1|2020-01-01 12:31:00|  bar|
#|         1|2020-01-01 12:32:00|  foo|
#|         1|2020-01-01 12:33:00|  foo|
#|         2|2020-01-01 13:00:00|  bar|
#|         2|2020-01-01 13:01:00|  foo|
#|         3|2020-01-01 13:02:00|  bar|
#|         3|2020-01-01 13:03:00|  foo|
#+----------+-------------------+-----+



+----------+-------------------+-----+
session_id| timestamp|event|
+----------+-------------------+-----+
 1|2020-01-01 12:31:00| bar|
 1|2020-01-01 12:32:00| foo|
 1|2020-01-01 12:33:00| foo|
 2|2020-01-01 13:00:00| bar|
 2|2020-01-01 13:01:00| foo|
 3|2020-01-01 13:02:00| bar|
 3|2020-01-01 13:03:00| foo|
+----------+-------------------+-----+

In [15]:
+----------+-----+
|session_id|event|
+----------+-----+
|         1|  bar|
|         1|  foo|
|         1|  foo|
|         2|  bar|
|         2|  foo|
|         3|  bar|
|         3|  foo|
+----------+-----+